In [1]:
import zipfile
import os, shutil
import collections
from pathlib import Path
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from keras.regularizers import l2
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, concatenate
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
import pylab as pl
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import itertools
import math


# audio lib
import librosa
import librosa.display
from librosa.util import fix_length

import IPython.display as ipd
from PIL import Image

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
output_folder = "/kaggle/working/melspectrogram_tmp/"
os.mkdir(output_folder)

# Classificazione Canzoni Utente

Una volta generato il modello si andrà a utilizzarlo per fare inferenza sul mood delle canzoni dell'utente. In questo caso si utilizzano come test canzoni scelte in maniera da coprire più generi e mood possibili. Il sistema andrà a generare per ogni canzone, divisa in blocchi di 6 secondi, vari spectrogrammi di Mel e si andrà a classificare ognuno di essi, arrivando ad avere per ogni canzone una percentuale del tipo di mood evocato, andando quindi a considerare in maniera più fine il mood totale della canzone.

In [4]:
base_net = load_model('../input/effnetb3spectrogram/weights.cnn.hdf5')
layer_name = 'flatten'
model = Model(inputs=base_net.input, outputs=base_net.get_layer(layer_name).output)

2022-06-22 15:00:58.751652: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 15:00:58.870847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 15:00:58.871703: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 15:00:58.873509: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
LSTM = load_model('../input/lstmspectrogram/weights.cnn.hdf5')

In [6]:
data = {
    'ID' : [],
    'A-V-' : [],
    'A-V+' : [],
    'A+V-' : [],
    'A+V+' : []
}

directory = '../input/my-playlist/Playlist/'

SAMPLE_RATE = 22050
TRACK_DURATION = 480
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

num_segments = 80
samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)


num_mel = 96
n_fft = 2048
hop_length = 512

num_mel_vectors_per_segment = math.ceil(samples_per_segment / hop_length)


In [7]:
print("Classificazione mood canzioni in corso...")
print("Legenda:")
print("0: basso Arousal - basso valence ")
print("1: basso Arousal - alto valence ")
print("2: alto Arousal - basso valence ")
print("3: alto Arousal - alto valence ")
print("----------------------------------------------------")

# per ogni file musicale genero uno spectogramma temporaneo
for filename in os.listdir(directory):
    labels = []
    
    filename_no_extension = Path(filename).stem
    data["ID"].append(filename_no_extension)
    
    file_path = os.path.join(directory, filename)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE,  offset=5.0, duration=TRACK_DURATION)

    # genero spectogrammi per ogni 6 secondi di canzone
    for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mel
                    mel = librosa.feature.melspectrogram(y=signal[start:finish], sr=sample_rate, n_mels=num_mel, hop_length=hop_length)
                    mel = mel.T

                    
                    if (len(mel) == num_mel_vectors_per_segment):
                        
                        
                        img_name = str(d) + '.png'

                        


                        fig = plt.figure(figsize=(3, 3), dpi=100)
                        S_dB = librosa.power_to_db(mel.T)
                        img = librosa.display.specshow(S_dB)
                        
                        plt.savefig("{}{}".format(output_folder,img_name))
                        plt.close(fig)

    # per ogni 4 spettrogrammi generati dalla canzone eseguo la predizione e creo una lista con tutte le predizioni riferite ad una canzone
    counter_spectrogram = 0
    pic_array = []
    features_spectrogram = []
    
    for i in os.listdir(output_folder):
        
        counter_spectrogram = counter_spectrogram + 1 # incremento il contatore di frame

        # ottengo le features
        file_path = os.path.join(output_folder, i)
        pic = image.load_img(file_path)
        pic = np.array(pic)
        pic_array.append(pic)

            
        if counter_spectrogram == 4:
            pic_array = np.array(pic_array)
            features_spectrogram = model.predict(pic_array)
            features_spectrogram = np.reshape(features_spectrogram, (1, 4, -1))
            classification = LSTM.predict(features_spectrogram)
            labels.append(np.argmax(classification))
            
            counter_spectrogram = 0
            pic_array = []
            features_spectrogram = []
    
    
    # creo una lista con le percentuali di mood per ogni canzone

    counter = collections.Counter(labels)
    most_common= [(i, round(counter[i] / len(labels) * 100.0, 1)) for i, count in counter.most_common()]
    
    # trasformo la lista in campi per il dataframe
    i0 = False
    i1 = False
    i2 = False
    i3 = False
    
    for i in range(len(most_common)):
        if most_common[i][0] == 0:
            i0 = True
            data['A-V-'].append(most_common[i][1])
        elif most_common[i][0] == 1:
            i1 = True
            data['A-V+'].append(most_common[i][1])
        elif most_common[i][0] == 2:
            i2 = True
            data['A+V-'].append(most_common[i][1])
        elif most_common[i][0] == 3:
            i3 = True
            data['A+V+'].append(most_common[i][1])
            
    
    if not(i0):
        data['A-V-'].append(0)
    if not(i1):
        data['A-V+'].append(0)
    if not(i2):
        data['A+V-'].append(0)
    if not(i3):
        data['A+V+'].append(0)

            
        
    

    print("File: "+filename)
    print("Predizioni:")
    print(labels)
    print("Mood in percentuale: "+ str(most_common))    
    print("----------------------------------------------------")
    
    # cancello la directory temporanea e la ricero
    shutil.rmtree(output_folder)
    os.mkdir(output_folder)

Classificazione mood canzioni in corso...
Legenda:
0: basso Arousal - basso valence 
1: basso Arousal - alto valence 
2: alto Arousal - basso valence 
3: alto Arousal - alto valence 
----------------------------------------------------


2022-06-22 15:01:20.191939: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-22 15:01:22.002571: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


File: Fabrizio De Andr - La canzone di Marinella.mp3
Predizioni:
[1, 1, 1, 1, 1, 1, 0]
Mood in percentuale: [(1, 85.7), (0, 14.3)]
----------------------------------------------------
File: John Coltrane - Naima.mp3
Predizioni:
[1, 0, 1, 0, 1, 0, 0, 1, 1, 0]
Mood in percentuale: [(1, 50.0), (0, 50.0)]
----------------------------------------------------
File: Nirvana - Come As You Are.mp3
Predizioni:
[3, 3, 3, 3, 2, 3, 1, 3, 3]
Mood in percentuale: [(3, 77.8), (2, 11.1), (1, 11.1)]
----------------------------------------------------
File: Queen - Radio Ga Ga.mp3
Predizioni:
[3, 2, 3, 3, 3, 3, 1, 3, 3, 0, 2, 3, 3, 3]
Mood in percentuale: [(3, 71.4), (2, 14.3), (1, 7.1), (0, 7.1)]
----------------------------------------------------
File: Red Hot Chili Peppers - By The Way.mp3
Predizioni:
[2, 0, 2, 2, 3, 2, 2, 2]
Mood in percentuale: [(2, 75.0), (0, 12.5), (3, 12.5)]
----------------------------------------------------
File: Imagine Dragons - On Top of the World.mp3
Predizioni:
[3, 3, 2

In [8]:
df = pd.DataFrame(data)
df.to_csv('mel.csv',index=False)

df

,ID,A-V-,A-V+,A+V-,A+V+
0,Fabrizio De Andr - La canzone di Marinella,14.3,85.7,0.0,0.0
1,John Coltrane - Naima,50.0,50.0,0.0,0.0
2,Nirvana - Come As You Are,0.0,11.1,11.1,77.8
3,Queen - Radio Ga Ga,7.1,7.1,14.3,71.4
4,Red Hot Chili Peppers - By The Way,12.5,0.0,75.0,12.5
5,Imagine Dragons - On Top of the World,0.0,0.0,14.3,85.7
6,Green Day - Time of Your Life,50.0,33.3,0.0,16.7
7,Oasis - Wonderwall,11.1,11.1,0.0,77.8
8,Eagles - Hotel California,25.0,43.8,0.0,31.2
9,Red Hot Chili Peppers - Snow,0.0,36.4,0.0,63.6
